In [1]:
'''SETUP for Using TensorFlow'''

# import sys
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# print(f"Tensor-Flow Version: {tf.__version__}")
# print(f"Python {sys.version}")
# gpu = len(tf.config.list_physical_devices('GPU'))>0
# print("GPU is", "available" if gpu else "NOT AVAILABLE")
# print(f"GPU Devices {tf.config.list_physical_devices('GPU')}")
# print(f"Setting dynamic growth of GPU to True")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)


'SETUP for Using TensorFlow'

In [2]:
import pennylane as qml
from pennylane import numpy as np

# F.1

## Fast Fourier Transformation


### Multiply two polynomials
Given two polynomials,$A(x) = 1 + x$ and $B(x) = 1 - x$, use FFT to multiply these two polynomials

In [3]:
# Step 1. 
coeff_rep_of_p1 = np.asarray([1,1])
coeff_rep_of_p2 = np.asarray([1,-1])
#Multiply two polynomialas using Fast Fourier Transform
def multiply_polynomials(p1,p2):
    # Step 2. 
    # Convert the polynomials to coefficients
    coeff_rep_of_p1 = np.asarray([1,1])
    coeff_rep_of_p2 = np.asarray([1,-1])
    # Step 3.
    # Multiply the coefficients
    coeff_rep_of_p1_times_p2 = np.convolve(coeff_rep_of_p1, coeff_rep_of_p2)
    # Step 4.
    # Convert the coefficients to polynomials
    p1_times_p2 = np.poly1d(coeff_rep_of_p1_times_p2)
    return p1_times_p2
p1 = np.poly1d([1,1])
p2 = np.poly1d([1,-1])
ans = multiply_polynomials(p1,p2)
print(ans)
ans




   2
1 x - 1


poly1d(([ 1,  0, -1], requires_grad=True)

### Codercise F.1.1. 
Given a polynomial in its coefficient representation, convert it into a value representation using NumPy's DFT/FFT module.

In [4]:
def coefficients_to_values(coefficients):
    """Returns the value representation of a polynomial
    
    Args:
        coefficients (array[complex]): a 1-D array of complex 
            coefficients of a polynomial with 
            index i representing the i-th degree coefficient

    Returns: 
        array[complex]: the value representation of the 
            polynomial 
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    val = np.fft.fft(coefficients)
    return val

A = [4, 3, 2, 1]
print(coefficients_to_values(A))

[10.+0.j  2.-2.j  2.+0.j  2.+2.j]


### Codercise F.1.2. 
Given a polynomial in its value representation, use the NumPy's DFT/FFT module to convert from the value representation to the coefficient representation.

In [5]:
def values_to_coefficients(values):
    """Returns the coefficient representation of a polynomial
    
    Args:
        values (array[complex]): a 1-D complex array with 
            the value representation of a polynomial 

    Returns: 
        array[complex]: a 1-D complex array of coefficients
    """
    
    ##################
    # YOUR CODE HERE #
    ################## 
    return np.fft.ifft(values)


A = [10.+0.j,  2.-2.j,  2.+0.j,  2.+2.j]
print(values_to_coefficients(A))

[4.+0.j 3.+0.j 2.+0.j 1.+0.j]


In [6]:
# calculates a power of 2 that is greater than a given number.
def next_power_of_2(n):
    """Returns the next power of 2 that is greater than a given number
    
    Args:
        n (int): a positive integer
    
    Returns: 
        int: the next power of 2 that is greater than n
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    return int(np.ceil(np.log2(n)))
next_power_of_2(18)

5

### Codercise F.1.3. 
Implement a helper function nearest_power_of_2 that calculates a power of 2 that is greater than a given number.

In [7]:
def nearest_power_of_2(x):
    """Given an integer, return the nearest power of 2. 
    
    Args:
        x (int): a positive integer

    Returns: 
        int: the nearest power of 2 of x
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    power = int(np.ceil(np.log2(x)))
    return 2**power
nearest_power_of_2(25)   


32

### Codercise F.1.4. 
Given two polynomials in their coefficient representation, write a function to multiply them both using the functions coefficients_to_values, nearest_power_of_2, and values_to_coefficients.

In [8]:
def fft_multiplication(poly_a, poly_b):
    """Returns the result of multiplying two polynomials
    
    Args:
        poly_a (array[complex]): 1-D array of coefficients 
        poly_b (array[complex]): 1-D array of coefficients 

    Returns: 
        array[complex]: complex coefficients of the product
            of the polynomials
    """
    ##################
    # YOUR CODE HERE #
    ################## 

    # Calculate the number of values required
    x = len(poly_a) + len(poly_b) + 5

    # Figure out the nearest power of 2
    nearest_power = nearest_power_of_2(x)
    roots = int(np.log2(nearest_power))

    # Pad zeros to the polynomial
    poly_a_paded = np.pad(poly_a,(0,roots - len(poly_a)),'constant', constant_values=(0+ 0.j))
    poly_b_paded = np.pad(poly_b,(0,roots - len(poly_b)),'constant', constant_values=(0 + 0.j))
    # Convert the polynomials to value representation 
    values_a = coefficients_to_values(poly_a_paded)
    values_b = coefficients_to_values(poly_b_paded)
    # Multiply
    ele_mult = np.multiply(values_a,values_b)

    # Convert back to coefficient representation
    coeff = values_to_coefficients(ele_mult)
    return coeff

In [9]:
poly_a = np.array([1 + 0.j,1+0.j,2 + 0.j])
poly_b = np.array([-1 + 0.j,1+0.j, 2 + 0.j,3 + 0.j])
print
print(fft_multiplication(poly_a,poly_b))

[6.+0.j 6.+0.j 1.+0.j 7.+0.j]


## Quantum Fourier transform

### Codercise F.2.1.
Implement the circuit that performs the single-qubit QFT operation, i.e., for $n = 1$.

In [10]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def one_qubit_QFT(basis_id):
    """A circuit that computes the QFT on a single qubit. 
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubit after applying QFT.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0])

    ##################
    # YOUR CODE HERE #
    ##################
    qml.Hadamard(wires = 0)
    return qml.state()
one_qubit_QFT(1)


tensor([ 0.70710678+0.j, -0.70710678+0.j], requires_grad=True)

### Codercise F.2.2. 
Implement a circuit that performs the two-qubit QFT operation. First, derive the QFT matrix for $n = 2$. Then, use the qml.QubitUnitary operation to apply it.

In [11]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def two_qubit_QFT(basis_id):
    """A circuit that computes the QFT on two qubits using qml.QubitUnitary. 
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    ##################
    # YOUR CODE HERE #
    ##################
    omega = 0 +1.j
    matrix_ = np.zeros((4,4), dtype=complex)
    for i in range(4):
        for j in range(4):
            if i == 0:
                matrix_[i,j] = 1
            elif j == 0:
                matrix_[i,j] = 1
            else:
                matrix_[i,j] = omega**(i*j)
    qml.QubitUnitary( 1/2 * matrix_,wires = [0,1])
    return qml.state()
            
two_qubit_QFT(3)  
    
    


tensor([ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j], requires_grad=True)

In [37]:
hadamard_gate = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])
s_gate = np.array([[1, 0], [0, 1j]])
hadamard_gate_s = np.tensordot(s_gate, hadamard_gate, axes=1)
U_QFT = [hadamard_gate,hadamard_gate,hadamard_gate_s, -1 * hadamard_gate_s] * 1/np.sqrt(2)



(4, 4)

### Codercise F.2.3. 
Implement the two-qubit QFT using a combination of gates (without using qml.QubitUnitary).

In [39]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def decompose_two_qubit_QFT(basis_id):
    """A circuit that computes the QFT on two qubits using elementary gates.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    ##################
    # YOUR CODE HERE #
    ##################
    # # qml.Hadamard(wires = 1)
    # qml.S(wires = 1)
    # qml.ctrl(qml.S,control = 1)(wires = 0)
    # qml.Hadamard(wires = 0)
    # return qml.state()
    qml.Hadamard(wires = 0)
    #qml.S(wires = 0)
    qml.ctrl(qml.S, control = 0)(wires = 1)
    qml.Hadamard(wires = 1)
    qml.SWAP(wires = [0,1])
    return qml.state()
decompose_two_qubit_QFT(3)

tensor([ 5.000000e-01+0.j , -3.061617e-17-0.5j, -5.000000e-01+0.j ,
         3.061617e-17+0.5j], requires_grad=True)

## F.3

## Codercise F.3.1. 
Implement the QFT for three qubits.



In [41]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    qml.Hadamard(wires = 0)
    qml.ctrl(qml.S,control = 0 )(wires = 1)
    qml.ctrl(qml.T,control = 0 )(wires = 2)
    qml.Hadamard(wires = 1)
    qml.ctrl(qml.S,control = 1 )(wires = 2)
    qml.Hadamard(wires = 2)
    qml.SWAP(wires = [0,2])
    return qml.state()
    ##################
    # YOUR CODE HERE #
    ################## 
three_qubit_QFT(7)



tensor([ 3.53553391e-01+0.j        ,  2.50000000e-01-0.25j      ,
        -2.16489014e-17-0.35355339j, -2.50000000e-01-0.25j      ,
        -3.53553391e-01+0.j        , -2.50000000e-01+0.25j      ,
         2.16489014e-17+0.35355339j,  2.50000000e-01+0.25j      ], requires_grad=True)

## Codercise F.3.2. 
Implement a circuit that reverses the order of $n$qubits using SWAP gates. You need only $n/2$ gates for this.

In [91]:
dev = qml.device('default.qubit', wires=4)

            
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    n_qubits = n_qubits
    print("Range:",n_qubits//2)
    for i in range(n_qubits//2):
        print(f"I: {i},wire1: {i}, wire2: {n_qubits-i-1}")
        qml.SWAP(wires = [i,n_qubits-i-1])

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    # qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()
qft_node(3,4)


Range: 2
I: 0,wire1: 0, wire2: 3
I: 1,wire1: 1, wire2: 2


tensor([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], requires_grad=True)

In [125]:
dev = qml.device('default.qubit', wires=4)

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    for qubit in range(n_qubits):
        #print(f"Hadamard on qubit {qubit}")
        qml.Hadamard(qubit)
        rotation_ = 2
        target_ = 1
        for q in range(qubit+1,n_qubits):
            #print(f"Source {qubit} Target {qubit+next_}")
            print(f"Rotation angle {rotation_} ,source {qubit} ,target {target_}")
            qml.ControlledPhaseShift((2*np.pi)/ 2**rotation_,wires = [qubit,qubit+target_])
            rotation_ += 1
            target_ += 1
        #print()

            
        
        

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


qft_node(7,4)

Rotation angle 2 ,source 0 ,target 1
Rotation angle 3 ,source 0 ,target 2
Rotation angle 4 ,source 0 ,target 3
Rotation angle 2 ,source 1 ,target 1
Rotation angle 3 ,source 1 ,target 2
Rotation angle 2 ,source 2 ,target 1
Range: 2
I: 0,wire1: 0, wire2: 3
I: 1,wire1: 1, wire2: 2


tensor([ 2.50000000e-01+0.j        , -2.30969883e-01+0.09567086j,
         1.76776695e-01-0.1767767j , -9.56708581e-02+0.23096988j,
        -1.53080850e-17-0.25j      ,  9.56708581e-02+0.23096988j,
        -1.76776695e-01-0.1767767j ,  2.30969883e-01+0.09567086j,
        -2.50000000e-01+0.j        ,  2.30969883e-01-0.09567086j,
        -1.76776695e-01+0.1767767j ,  9.56708581e-02-0.23096988j,
         1.53080850e-17+0.25j      , -9.56708581e-02-0.23096988j,
         1.76776695e-01+0.1767767j , -2.30969883e-01-0.09567086j], requires_grad=True)

### Codercise F.3.5. 
Implement the QFT using qml.QFT.



In [127]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def pennylane_qft(basis_id, n_qubits):
    """A that circuit performs the QFT using PennyLane's QFT template.
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
        n_qubits (int): An integer identifying the 
            number of qubits.
            
    Returns:
        array[complex]: The state after applying the QFT to the qubits.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))

    ##################
    # YOUR CODE HERE #
    ################## 
    qml.QFT(wires=range(n_qubits))
    return qml.state()
pennylane_qft(3, 4)

tensor([ 2.50000000e-01+0.00000000e+00j,  9.56708581e-02+2.30969883e-01j,
        -1.76776695e-01+1.76776695e-01j, -2.30969883e-01-9.56708581e-02j,
         1.66533454e-16-2.50000000e-01j,  2.30969883e-01-9.56708581e-02j,
         1.76776695e-01+1.76776695e-01j, -9.56708581e-02+2.30969883e-01j,
        -2.50000000e-01-3.33066907e-16j, -9.56708581e-02-2.30969883e-01j,
         1.76776695e-01-1.76776695e-01j,  2.30969883e-01+9.56708581e-02j,
        -4.99600361e-16+2.50000000e-01j, -2.30969883e-01+9.56708581e-02j,
        -1.76776695e-01-1.76776695e-01j,  9.56708581e-02-2.30969883e-01j], requires_grad=True)